## basic workings

In [ ]:
!pip install 

     |████████████████████████████████| 2.6 MB 13.2 MB/s 
     |████████████████████████████████| 3.3 MB 46.7 MB/s 
     |████████████████████████████████| 636 kB 52.7 MB/s 
     |████████████████████████████████| 895 kB 47.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, AutoModel, pipeline

import torch
from torch.nn import functional as F

In [ ]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Biocreative/Biocreative/litcovid_dataset.csv')

In [ ]:
sample_row = data.iloc[]
sample_row

pmid                                                             32661141
journal                                                           J Virol
title                   Metabolic Syndrome and Viral Pathogenesis: Les...
abstract                Metabolic syndrome increases the risk of sever...
keywords                covid-19;sars-cov-2;diabetes;influenza;metabol...
pub_type                Journal Article;Research Support, N.I.H., Extr...
authors                 Smith, Maria;Honce, Rebekah;Schultz-Cherry, St...
doi                                                  10.1128/JVI.00665-20
label                                                 Treatment;Mechanism
Case Report                                                             0
Diagnosis                                                               0
Epidemic Forecasting                                                    0
Mechanism                                                               1
Prevention                            

In [ ]:
sample_item = sample_row['abstract']
sample_labels = sample_row.to_dict()
labels = list(sample_labels.keys())[9:]
print(labels, sample_item)

['Case Report', 'Diagnosis', 'Epidemic Forecasting', 'Mechanism', 'Prevention', 'Transmission', 'Treatment'] Metabolic syndrome increases the risk of severe disease due to viral infection. Yet few studies have assessed the pathogenesis of respiratory viruses in high-risk populations. Here, we summarize how metabolic dysregulation impairs immune responses, and we define the role of metabolism during influenza virus and coronavirus infections. We also discuss the use of various in vitro, in vivo, and ex vivo models to elucidate the contributions of host factors to viral susceptibility, immunity, and disease severity.


In [ ]:
classifier(sample_item, labels)

{'labels': ['Mechanism',
  'Diagnosis',
  'Case Report',
  'Transmission',
  'Epidemic Forecasting',
  'Prevention',
  'Treatment'],
 'scores': [0.24647271633148193,
  0.1775263249874115,
  0.15522360801696777,
  0.12992294132709503,
  0.12695592641830444,
  0.0996629074215889,
  0.06423550099134445],
 'sequence': 'Metabolic syndrome increases the risk of severe disease due to viral infection. Yet few studies have assessed the pathogenesis of respiratory viruses in high-risk populations. Here, we summarize how metabolic dysregulation impairs immune responses, and we define the role of metabolism during influenza virus and coronavirus infections. We also discuss the use of various in vitro, in vivo, and ex vivo models to elucidate the contributions of host factors to viral susceptibility, immunity, and disease severity.'}

In [ ]:
MODEL_NAME = 'deepset/sentence_bert'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at deepset/sentence_bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
data = pd.read_csv('drive/MyDrive/Biocreative/Biocreative/litcovid_dataset.csv')
data = data.drop(['pmid', 'journal', 'title', 'keywords', 'pub_type', 'authors', 'doi', 'label'], axis=1)
data.head()

,abstract,Case Report,Diagnosis,Epidemic Forecasting,Mechanism,Prevention,Transmission,Treatment
0,"In December 2019, a new and highly contagious ...",0,0,0,1,0,0,1
1,"A novel coronavirus disease (COVID-19), transm...",0,0,0,0,1,0,1
2,"BACKGROUND: In December 2019, the novel corona...",1,0,0,0,0,0,0
3,During the coronavirus disease 2019 (COVID-19)...,0,0,0,0,1,0,0
4,OBJECTIVES: Sofosbuvir and daclatasvir are dir...,0,0,0,0,0,0,1


In [ ]:
LABEL_COLUMNS = data.columns[1:]
Sample_row = data.iloc[1]
Sample_abstract = Sample_row.abstract
Sample_labels = Sample_row[LABEL_COLUMNS]

In [ ]:
true_labels = [key for key, value in Sample_labels.to_dict().items() if value == 1]
true_labels

['Prevention', 'Treatment']

In [ ]:
Sample_abstract

'A novel coronavirus disease (COVID-19), transmitted from humans to humans, has rapidly become the pandemic responsible for the current global health crisis. COVID-19 is caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2), which is said to be of zoonotic origin. This review describes the etiology and signs and symptoms as well as the current allopathic therapy for COVID-19. Additionally, findings of previous studies on the immunomodulatory effects and antiviral activities of particular foods and herbs on influenza virus and coronaviruses have been collated, with the aim of promoting the use of dietary therapy and herbal medicine as COVID-19 preventive therapies, while specific drugs and vaccines are yet to be discovered or are still under development. The volume of existing reports is irrefutable evidence that foods and herbs possess a potential antiviral ability against SARS-CoV-2 and can prevent COVID-19. Foods and herbs could be used as dietary or complementary th

In [ ]:
encoding = tokenizer.batch_encode_plus(
    [Sample_abstract] + true_labels,
    return_tensors="pt",
    pad_to_max_length=True
) 

encoding.keys()

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
output = model(encoding["input_ids"], encoding["attention_mask"])[0]

In [ ]:
sentence_rep = output[:1].mean(dim=1)
label_reps = output[1:].mean(dim=1)

In [ ]:
similarities = F.cosine_similarity(sentence_rep, label_reps)
closest = similarities.argsort(descending=True)
for ind in closest:
    print(f'label: {true_labels[ind]} \t similarity: {similarities[ind]}')

label: Treatment 	 similarity: 0.24514493346214294
label: Prevention 	 similarity: 0.17372678220272064


## main model

In [11]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')  
nltk.download('punkt')
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('drive/MyDrive/Biocreative/Biocreative/litcovid_pro.csv')
data.head()

,abstract,Case Report,Diagnosis,Epidemic Forecasting,Mechanism,Prevention,Transmission,Treatment
0,"In December 2019, a new and highly contagious ...",0,0,0,1,0,0,1
1,"A novel coronavirus disease (COVID-19), transm...",0,0,0,0,1,0,1
2,"BACKGROUND: In December 2019, the novel corona...",1,0,0,0,0,0,0
3,During the coronavirus disease 2019 (COVID-19)...,0,0,0,0,1,0,0
4,OBJECTIVES: Sofosbuvir and daclatasvir are dir...,0,0,0,0,0,0,1


In [6]:
example = data.abstract[4]
print(example, len(example.split(' ')))

OBJECTIVES: Sofosbuvir and daclatasvir are direct-acting antivirals highly effective against hepatitis C virus. There is some in silico and in vitro evidence that suggests these agents may also be effective against SARS-CoV-2. This trial evaluated the effectiveness of sofosbuvir in combination with daclatasvir in treating patients with COVID-19. METHODS: Patients with a positive nasopharyngeal swab for SARS-CoV-2 on RT-PCR or bilateral multi-lobar ground-glass opacity on their chest CT and signs of severe COVID-19 were included. Subjects were divided into two arms with one arm receiving ribavirin and the other receiving sofosbuvir/daclatasvir. All participants also received the recommended national standard treatment which, at that time, was lopinavir/ritonavir and single-dose hydroxychloroquine. The primary endpoint was time from starting the medication until discharge from hospital with secondary endpoints of duration of ICU stay and mortality. RESULTS: Sixty-two subjects met the inc

In [12]:
stop_words = set(stopwords.words('english'))

word_tokens = word_tokenize(example)
 
filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

In [13]:
print(word_tokens)
print(filtered_sentence)

['OBJECTIVES', ':', 'Sofosbuvir', 'and', 'daclatasvir', 'are', 'direct-acting', 'antivirals', 'highly', 'effective', 'against', 'hepatitis', 'C', 'virus', '.', 'There', 'is', 'some', 'in', 'silico', 'and', 'in', 'vitro', 'evidence', 'that', 'suggests', 'these', 'agents', 'may', 'also', 'be', 'effective', 'against', 'SARS-CoV-2', '.', 'This', 'trial', 'evaluated', 'the', 'effectiveness', 'of', 'sofosbuvir', 'in', 'combination', 'with', 'daclatasvir', 'in', 'treating', 'patients', 'with', 'COVID-19', '.', 'METHODS', ':', 'Patients', 'with', 'a', 'positive', 'nasopharyngeal', 'swab', 'for', 'SARS-CoV-2', 'on', 'RT-PCR', 'or', 'bilateral', 'multi-lobar', 'ground-glass', 'opacity', 'on', 'their', 'chest', 'CT', 'and', 'signs', 'of', 'severe', 'COVID-19', 'were', 'included', '.', 'Subjects', 'were', 'divided', 'into', 'two', 'arms', 'with', 'one', 'arm', 'receiving', 'ribavirin', 'and', 'the', 'other', 'receiving', 'sofosbuvir/daclatasvir', '.', 'All', 'participants', 'also', 'received', 'th

In [16]:
len(filtered_sentence)

188

In [18]:
example

'OBJECTIVES: Sofosbuvir and daclatasvir are direct-acting antivirals highly effective against hepatitis C virus. There is some in silico and in vitro evidence that suggests these agents may also be effective against SARS-CoV-2. This trial evaluated the effectiveness of sofosbuvir in combination with daclatasvir in treating patients with COVID-19. METHODS: Patients with a positive nasopharyngeal swab for SARS-CoV-2 on RT-PCR or bilateral multi-lobar ground-glass opacity on their chest CT and signs of severe COVID-19 were included. Subjects were divided into two arms with one arm receiving ribavirin and the other receiving sofosbuvir/daclatasvir. All participants also received the recommended national standard treatment which, at that time, was lopinavir/ritonavir and single-dose hydroxychloroquine. The primary endpoint was time from starting the medication until discharge from hospital with secondary endpoints of duration of ICU stay and mortality. RESULTS: Sixty-two subjects met the in

In [40]:
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

for idx, ele in enumerate(filtered_sentence):
  if ele in punc:
    del filtered_sentence[idx]

In [28]:
' '.join(filtered_sentence)

'OBJECTIVES Sofosbuvir daclatasvir direct-acting antivirals highly effective hepatitis C virus silico vitro evidence suggests agents may also effective SARS-CoV-2 trial evaluated effectiveness sofosbuvir combination daclatasvir treating patients COVID-19 METHODS Patients positive nasopharyngeal swab SARS-CoV-2 RT-PCR bilateral multi-lobar ground-glass opacity chest CT signs severe COVID-19 included Subjects divided two arms one arm receiving ribavirin receiving sofosbuvir/daclatasvir participants also received recommended national standard treatment time lopinavir/ritonavir single-dose hydroxychloroquine primary endpoint time starting medication discharge hospital secondary endpoints duration ICU stay mortality RESULTS Sixty-two subjects met inclusion criteria 35 enrolled sofosbuvir/daclatasvir arm 27 ribavirin arm median duration stay 5 days sofosbuvir/daclatasvir group 9 days ribavirin group mortality sofosbuvir/daclatasvir group 2/35 6  9/27 33  ribavirin group relative risk death p

In [29]:
len(filtered_sentence)

155

In [ ]:
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')  

lemmatizer = WordNetLemmatizer()

In [41]:
new_list = [lemmatizer.lemmatize(word) for word in filtered_sentence]

In [42]:
len(new_list)

152

In [44]:
' '.join(new_list) 

'OBJECTIVES Sofosbuvir daclatasvir direct-acting antiviral highly effective hepatitis C virus silico vitro evidence suggests agent may also effective SARS-CoV-2 trial evaluated effectiveness sofosbuvir combination daclatasvir treating patient COVID-19 METHODS Patients positive nasopharyngeal swab SARS-CoV-2 RT-PCR bilateral multi-lobar ground-glass opacity chest CT sign severe COVID-19 included Subjects divided two arm one arm receiving ribavirin receiving sofosbuvir/daclatasvir participant also received recommended national standard treatment time lopinavir/ritonavir single-dose hydroxychloroquine primary endpoint time starting medication discharge hospital secondary endpoint duration ICU stay mortality RESULTS Sixty-two subject met inclusion criterion 35 enrolled sofosbuvir/daclatasvir arm 27 ribavirin arm median duration stay 5 day sofosbuvir/daclatasvir group 9 day ribavirin group mortality sofosbuvir/daclatasvir group 2/35 6 9/27 33 ribavirin group relative risk death patient trea

In [54]:
def remove_stopword(sentence):

  punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

  word_tokens = word_tokenize(sentence)
  filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

  for idx, ele in enumerate(filtered_sentence):
    if ele in punc:
      del filtered_sentence[idx]

  new_list = [lemmatizer.lemmatize(word) for word in filtered_sentence]

  return ' '.join(new_list)

In [61]:
df1 = data['abstract']
df2 = df1.apply(remove_stopword)

In [70]:
df3 = pd.concat([df2, data.drop(["abstract"], axis=1)], axis=1)

In [72]:
df3.head()

,abstract,Case Report,Diagnosis,Epidemic Forecasting,Mechanism,Prevention,Transmission,Treatment
0,December 2019 new highly contagious infectious...,0,0,0,1,0,0,1
1,"novel coronavirus disease COVID-19 , transmitt...",0,0,0,0,1,0,1
2,BACKGROUND December 2019 novel coronavirus SAR...,1,0,0,0,0,0,0
3,coronavirus disease 2019 COVID-19 pandemic imp...,0,0,0,0,1,0,0
4,OBJECTIVES Sofosbuvir daclatasvir direct-actin...,0,0,0,0,0,0,1


In [73]:
df3.to_csv('/content/drive/MyDrive/Biocreative/Biocreative/preprocessed.csv', index=False)

In [74]:
df = pd.read_csv('/content/drive/MyDrive/Biocreative/Biocreative/preprocessed.csv')
df.head()

,abstract,Case Report,Diagnosis,Epidemic Forecasting,Mechanism,Prevention,Transmission,Treatment
0,December 2019 new highly contagious infectious...,0,0,0,1,0,0,1
1,"novel coronavirus disease COVID-19 , transmitt...",0,0,0,0,1,0,1
2,BACKGROUND December 2019 novel coronavirus SAR...,1,0,0,0,0,0,0
3,coronavirus disease 2019 COVID-19 pandemic imp...,0,0,0,0,1,0,0
4,OBJECTIVES Sofosbuvir daclatasvir direct-actin...,0,0,0,0,0,0,1
